# Privacy-Preserving RAG: Clinical Assistant with Tool Calling

This notebook implements a clinical assistant powered by OpenAI's O3 model that can answer patient-specific questions using structured data while preserving privacy through local de-identification.

## System Overview
- **OpenAI O3 Agent**: Acts as orchestrator and QA engine
- **Tool Functions**: Retrieve patient data from various sources
- **Local De-identifier**: Removes PHI before sending to OpenAI
- **Privacy-Safe Responses**: Clinically informed answers without compromising privacy

In [1]:
import os
import pandas as pd

def generate_note_auto(table_name, patient_id):
    """
    Reads the generated *_notes.csv file for the given table and returns 
    the first clinical note found for the provided patient_id.

    Parameters:
    - table_name: str (e.g., "observations", "medications")
    - patient_id: str (the patient_id to search for)

    Returns:
    - str: the clinical note or a message if not found
    """
    notes_file = os.path.join("data", f"{table_name}_notes.csv")

    if not os.path.exists(notes_file):
        return f"Note file not found: {notes_file}"

    df = pd.read_csv(notes_file)
    
    # Normalize column names just in case
    df.columns = [col.strip().lower() for col in df.columns]

    if "patient_id" not in df.columns or "clinical_note" not in df.columns:
        return "Expected columns not found in the CSV."

    matching_notes = df[df["patient_id"] == patient_id]

    if matching_notes.empty:
        return f"No note found for patient_id: {patient_id}"

    return matching_notes.iloc[0]["clinical_note"]

In [33]:
# Install essential packages
# Run this cell first to install core dependencies

!pip install openai python-dotenv pandas numpy

In [2]:
# Import necessary libraries
import os
import json
from typing import Dict, List, Any, Optional, Union
from datetime import datetime, date
import pandas as pd
import numpy as np

# OpenAI imports
from openai import OpenAI

# Environment variables
from dotenv import load_dotenv
load_dotenv()

print("✅ Libraries imported successfully!")
print(f"📅 Current date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

✅ Libraries imported successfully!
📅 Current date: 2025-08-06 22:05:22


In [4]:
# OpenAI Configuration
class ClinicalAssistantConfig:
    """Configuration for the Clinical Assistant"""
    
    def __init__(self):
        # OpenAI API configuration
        self.api_key = os.getenv("OPENAI_API_KEY")
        if not self.api_key:
            print("⚠️  Warning: OPENAI_API_KEY not found in environment variables")
            print("💡 Please set your OpenAI API key in a .env file or environment variable")
        
        # Model configuration - using GPT-4o-mini for now (O3 models may not be available yet)
        self.model = "gpt-4o-mini"  # Using GPT-4o-mini until O3 is available
        self.max_completion_tokens = 4000  # Use max_completion_tokens for newer models
        self.temperature = 0.1  # Low temperature for clinical accuracy
        
        # Tool calling configuration
        self.max_tool_calls = 5
        self.parallel_tool_calls = True

# Initialize configuration
config = ClinicalAssistantConfig()

# Initialize OpenAI client
if config.api_key:
    client = OpenAI(api_key=config.api_key)
    print("✅ OpenAI client initialized successfully!")
    print(f"🤖 Model: {config.model}")
else:
    client = None
    print("❌ OpenAI client not initialized - API key required")

✅ OpenAI client initialized successfully!
🤖 Model: gpt-4o-mini


In [5]:
# Tool Function Definitions for OpenAI Function Calling
# These will be used by the O3 model to understand available tools

TOOL_DEFINITIONS = [
    {
        "type": "function",
        "function": {
            "name": "get_patient_observations",
            "description": "Retrieve laboratory test results and clinical observations for a patient",
            "parameters": {
                "type": "object",
                "properties": {
                    "patient_id": {
                        "type": "string",
                        "description": "Unique patient identifier"
                    }
                },
                "required": ["patient_id"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_patient_conditions",
            "description": "Retrieve patient diagnosis information, medical conditions, and medical history",
            "parameters": {
                "type": "object",
                "properties": {
                    "patient_id": {
                        "type": "string",
                        "description": "Unique patient identifier"
                    }
                },
                "required": ["patient_id"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_patient_medications",
            "description": "Retrieve current and past medications for a patient",
            "parameters": {
                "type": "object",
                "properties": {
                    "patient_id": {
                        "type": "string",
                        "description": "Unique patient identifier"
                    }
                },
                "required": ["patient_id"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_patient_careplans",
            "description": "Retrieve care plans and treatment plans for a patient",
            "parameters": {
                "type": "object",
                "properties": {
                    "patient_id": {
                        "type": "string",
                        "description": "Unique patient identifier"
                    }
                },
                "required": ["patient_id"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_patient_procedures",
            "description": "Retrieve medical procedures and interventions performed on a patient",
            "parameters": {
                "type": "object",
                "properties": {
                    "patient_id": {
                        "type": "string",
                        "description": "Unique patient identifier"
                    }
                },
                "required": ["patient_id"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_patient_imaging_studies",
            "description": "Retrieve imaging studies and radiology reports for a patient",
            "parameters": {
                "type": "object",
                "properties": {
                    "patient_id": {
                        "type": "string",
                        "description": "Unique patient identifier"
                    }
                },
                "required": ["patient_id"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_patient_immunizations",
            "description": "Retrieve vaccination history and immunization records for a patient",
            "parameters": {
                "type": "object",
                "properties": {
                    "patient_id": {
                        "type": "string",
                        "description": "Unique patient identifier"
                    }
                },
                "required": ["patient_id"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_patient_allergies",
            "description": "Retrieve allergy information and adverse reactions for a patient",
            "parameters": {
                "type": "object",
                "properties": {
                    "patient_id": {
                        "type": "string",
                        "description": "Unique patient identifier"
                    }
                },
                "required": ["patient_id"]
            }
        }
    }
]

print("✅ Tool function definitions created!")
print(f"🔧 Available tools: {len(TOOL_DEFINITIONS)}")
for tool in TOOL_DEFINITIONS:
    print(f"   - {tool['function']['name']}: {tool['function']['description']}")

✅ Tool function definitions created!
🔧 Available tools: 8
   - get_patient_observations: Retrieve laboratory test results and clinical observations for a patient
   - get_patient_conditions: Retrieve patient diagnosis information, medical conditions, and medical history
   - get_patient_medications: Retrieve current and past medications for a patient
   - get_patient_careplans: Retrieve care plans and treatment plans for a patient
   - get_patient_procedures: Retrieve medical procedures and interventions performed on a patient
   - get_patient_imaging_studies: Retrieve imaging studies and radiology reports for a patient
   - get_patient_immunizations: Retrieve vaccination history and immunization records for a patient
   - get_patient_allergies: Retrieve allergy information and adverse reactions for a patient


In [ ]:
# gated model login with Hugging Face CLI
# Make sure you have the Hugging Face CLI installed and authenticated
!pip install huggingface_hub
!pip install llama-cpp-python

In [ ]:


# from huggingface_hub import login
# login(os.getenv("HUGGING_FACE_API_KEY"))

# from llama_cpp import Llama

# # Download the model to 'models/gemma-3-4b-it-qat-q4_0.gguf' as per previous steps
# llm = Llama.from_pretrained(
#     repo_id="google/gemma-3-4b-it-qat-q4_0-gguf",
#     filename="gemma-3-4b-it-q4_0.gguf",  # ✅ correct filename
#     n_ctx=2048,
#     n_threads=8
# )

/opt/anaconda3/envs/health-hw7/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
llama_model_load_from_file_impl: using device Metal (Apple M4 Max) - 49151 MiB free
llama_model_loader: loaded meta data with 39 key-value pairs and 444 tensors from /Users/aayush/.cache/huggingface/hub/models--google--gemma-3-4b-it-qat-q4_0-gguf/snapshots/15f73f5eee9c28f53afefef5723e29680c2fc78a/./gemma-3-4b-it-q4_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = gemma3
llama_model_loader: - kv   1:                      gemma3.context_length u32              = 131072
llama_model_loader: - kv   2:                         gemma3.block_count u32            

In [41]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from peft import PeftModel
import torch

BASE_MODEL = "google/gemma-2b"
LORA_MODEL_PATH = "./gemma-deid-lora/checkpoint-60"

# Load tokenizer and base model
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    torch_dtype=torch.float16,
    device_map="auto"
)

# Apply LoRA weights
model = PeftModel.from_pretrained(base_model, LORA_MODEL_PATH)
model.eval()


def deidentify(user_data: str):
    # Combine system prompt and user data
    prompt = f"""
        You are a local language model responsible for enforcing HIPAA compliance by identifying and 
        removing all Protected Health Information (PHI) from clinical text and structured data before 
        it is passed to an external system. Your task is to remove all 18 identifiers defined under 
        HIPAA's Safe Harbor rule while preserving the clinical meaning of the data.

        Redact all identifiers like names, dates, addresses, SSNs, etc. with placeholder [REDACTED], without summarizing or altering clinical facts.

        ---
        <data_with_phi>
        {user_data}
        </data_with_phi>
        <data_hipaa_compliant>
    """

    # Tokenize input
    device = "mps" if torch.mps.is_available() else "cpu"
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

    with torch.no_grad():
        output = model.generate(
            input_ids,
            max_new_tokens=300,         # ⬆️ Allow longer outputs
            temperature=0.7,
            do_sample=False,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id  # 👈 Avoid warnings
        )

    # Decode and extract only redacted portion
    decoded = tokenizer.decode(output[0], skip_special_tokens=True)

    if "<data_hipaa_compliant>" in decoded:
        redacted_part = decoded.split("<data_hipaa_compliant>")[-1]
        redacted_part = redacted_part.split("</data_hipaa_compliant>")[0].strip()
    else:
        redacted_part = decoded[len(prompt):].strip()

    print("🔒 Redaction complete. PHI has been removed from the clinical note.")
    print(f"📄 Redacted note: {redacted_part}")  # Print first 100 chars for preview
    return redacted_part

Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.11s/it]


In [43]:
# Tool Function Implementations (Using generate_notes_for_type)
class ClinicalDataTools:
    """Clinical data retrieval tools for the OpenAI assistant"""
    
    def __init__(self):
        self.de_identifier = None  # Will be initialized later
        print("🔧 Clinical Data Tools initialized with all available note types")
    
    def get_patient_observations(self, patient_id: str) -> str:
        """
        Retrieve laboratory test results and clinical observations for a patient
        
        Args:
            patient_id: Unique patient identifier
        """
        return generate_note_auto("observations",patient_id)
    
    def get_patient_conditions(self, patient_id: str) -> str:
        """
        Retrieve patient diagnosis information, medical conditions, and medical history
        
        Args:
            patient_id: Unique patient identifier
        """
        return generate_note_auto("conditions", patient_id)

    def get_patient_medications(self, patient_id: str) -> str:
        """
        Retrieve current and past medications for a patient
        
        Args:
            patient_id: Unique patient identifier
        """
        return generate_note_auto("medications", patient_id)

    def get_patient_careplans(self, patient_id: str) -> str:
        """
        Retrieve care plans and treatment plans for a patient
        
        Args:
            patient_id: Unique patient identifier
        """
        return generate_note_auto("careplans", patient_id)

    def get_patient_procedures(self, patient_id: str) -> str:
        """
        Retrieve medical procedures and interventions performed on a patient
        
        Args:
            patient_id: Unique patient identifier
        """
        return generate_note_auto("procedures", patient_id)

    def get_patient_imaging_studies(self, patient_id: str) -> str:
        """
        Retrieve imaging studies and radiology reports for a patient
        
        Args:
            patient_id: Unique patient identifier
        """
        return generate_note_auto("imaging_studies", patient_id)

    def get_patient_immunizations(self, patient_id: str) -> str:
        """
        Retrieve vaccination history and immunization records for a patient
        
        Args:
            patient_id: Unique patient identifier
        """
        return generate_note_auto("immunizations", patient_id)

    def get_patient_allergies(self, patient_id: str) -> str:
        """
        Retrieve allergy information and adverse reactions for a patient
        
        Args:
            patient_id: Unique patient identifier
        """
        return generate_note_auto("allergies", patient_id)

# Initialize the tools
clinical_tools = ClinicalDataTools()
print("✅ Clinical tools instance created with all available note types!")
print("📋 Available note types: observations, conditions, medications, careplans, procedures, imaging_studies, immunizations, allergies")

🔧 Clinical Data Tools initialized with all available note types
✅ Clinical tools instance created with all available note types!
📋 Available note types: observations, conditions, medications, careplans, procedures, imaging_studies, immunizations, allergies


In [44]:
# LLM Call and Prompt Management Functions
def build_system_prompt() -> str:
    """Build the system prompt for the clinical assistant"""
    return f"""You are an AI Clinical Reasoning Assistant with expertise in internal medicine. 
    Analyze clinical data and respond to patient-specific questions using structured reasoning.

    Available Tools:
    - get_patient_observations(): Laboratory test results and clinical observations
    - get_patient_conditions(): Diagnosis history, medical conditions, and medical history
    - get_patient_medications(): Current and past medications
    - get_patient_careplans(): Care plans and treatment plans
    - get_patient_procedures(): Medical procedures and interventions
    - get_patient_imaging_studies(): Imaging studies and radiology reports
    - get_patient_immunizations(): Vaccination history and immunization records
    - get_patient_allergies(): Allergy information and adverse reactions

    Clinical Reasoning Framework:
    1. Analyze the question to determine needed information
    2. Use appropriate tools to gather relevant patient data
    3. Synthesize findings from multiple data sources
    4. Provide clear, evidence-based responses with clinical reasoning

    Rules:
    - Use ONLY data provided by tool outputs
    - Reference relative timeframes when provided (e.g., "Day 0", "Post-op Day 3")
    - Acknowledge limitations if data is insufficient
    - Suggest additional information needed when applicable
    - Consider interactions between medications, conditions, and procedures
    - Always check for allergies before recommending treatments

    Current date: {datetime.now().strftime('%Y-%m-%d')}
    """

def build_messages(query: str, patient_id: str = None) -> List[Dict[str, str]]:
    """Build messages for the LLM call"""
    messages = [
        {"role": "system", "content": build_system_prompt()},
        {"role": "user", "content": f"Clinical query: {query}"}
    ]
    
    if patient_id:
        messages[-1]["content"] += f"\\nPatient ID: {patient_id}"
    
    return messages

def call_llm(client: OpenAI, config: ClinicalAssistantConfig, messages: List[Dict[str, str]]):
    """Make the actual LLM call"""
    return client.chat.completions.create(
        model=config.model,
        messages=messages,
        tools=TOOL_DEFINITIONS,
        tool_choice="auto",
        max_completion_tokens=config.max_completion_tokens,
        temperature=config.temperature
    )

In [45]:
# Clinical Assistant with Optional De-identification
class ClinicalAssistant:
    """Main clinical assistant that orchestrates OpenAI GPT-4o-mini model with tool calling"""
    
    def __init__(self, client: OpenAI, config: ClinicalAssistantConfig, tools: ClinicalDataTools):
        self.client = client
        self.config = config
        self.tools = tools
        
        # Map function names to actual methods
        self.function_map = {
            "get_patient_observations": self.tools.get_patient_observations,
            "get_patient_conditions": self.tools.get_patient_conditions,
            "get_patient_medications": self.tools.get_patient_medications,
            "get_patient_careplans": self.tools.get_patient_careplans,
            "get_patient_procedures": self.tools.get_patient_procedures,
            "get_patient_imaging_studies": self.tools.get_patient_imaging_studies,
            "get_patient_immunizations": self.tools.get_patient_immunizations,
            "get_patient_allergies": self.tools.get_patient_allergies
        }
    
    def de_identify_data(self, data: str) -> str:
        """De-identify text data using the existing deidentify function"""
        try:
            return deidentify(data)
        except Exception as e:
            print(f"Warning: De-identification failed: {e}")
            return data
    
    def execute_tool_call(self, function_name: str, arguments: Dict[str, Any], apply_deidentification: bool = False) -> str:
        """Execute a tool function call with optional de-identification"""
        if function_name not in self.function_map:
            return f"Error: Unknown function: {function_name}"
        print(f"Executing tool call: {function_name}")
        try:
            func = self.function_map[function_name]
            raw_result = func(**arguments)
            
            # Apply de-identification if requested
            if apply_deidentification and isinstance(raw_result, str):
                return self.de_identify_data(raw_result)
            
            return raw_result
            
        except Exception as e:
            return f"Error executing {function_name}: {str(e)}"
    
    def process_clinical_query(self, query: str, patient_id: str = None, apply_deidentification: bool = False) -> str:
        """
        Process a clinical query using OpenAI GPT-4o-mini with tool calling
        
        Args:
            query: The clinical question to answer
            patient_id: Optional patient ID if known
            apply_deidentification: Whether to apply de-identification to tool outputs
            
        Returns:
            Clinical assistant response
        """
        if not self.client:
            return "❌ OpenAI client not initialized. Please check your API key."
        
        try:
            # Get initial response from LLM
            messages = build_messages(query, patient_id)
            response = call_llm(self.client, self.config, messages)
            
            # Handle tool calls if any
            if response.choices[0].message.tool_calls:
                current_messages = messages.copy()
                current_messages.append({
                    "role": "assistant",
                    "content": response.choices[0].message.content,
                    "tool_calls": response.choices[0].message.tool_calls
                })
                
                # Process each tool call
                for tool_call in response.choices[0].message.tool_calls:
                    function_name = tool_call.function.name
                    function_args = json.loads(tool_call.function.arguments)
                    
                    # Execute tool call with optional de-identification
                    tool_result = self.execute_tool_call(function_name, function_args, apply_deidentification)
                    
                    current_messages.append({
                        "role": "tool",
                        "tool_call_id": tool_call.id,
                        "content": tool_result
                    })
                
                # Get final response with tool results
                final_response = call_llm(self.client, self.config, current_messages)
                return final_response.choices[0].message.content
            else:
                return response.choices[0].message.content
            
        except Exception as e:
            return f"❌ Error processing query: {str(e)}"

# Initialize the clinical assistant
if client:
    assistant = ClinicalAssistant(client, config, clinical_tools)
    print("✅ Clinical Assistant initialized with all 8 tool functions!")
else:
    assistant = None

✅ Clinical Assistant initialized with all 8 tool functions!


In [46]:
# Test Clinical Assistant with Integrated De-identification
print("🧪 Testing Clinical Assistant with Integrated De-identification")
print("=" * 60)

# Sample clinical queries to test the system with all available tools
sample_queries = [
    "What are this patient's current lab abnormalities and what do they suggest clinically?",
    "Can you summarize this patient's medical conditions and current medications?",
    "What procedures has this patient undergone and what was the care plan?",
    "Does this patient have any allergies I should be aware of before prescribing?",
    "What is this patient's vaccination status and immunization history?",
    "What imaging studies have been performed and what were the findings?",
    "Based on all available data, what is the comprehensive clinical picture?",
    "Are there any drug interactions or contraindications based on current medications and allergies?"
]

# Enhanced interactive testing function with de-identification options
# updated to use IPython display for better formatting
from IPython.display import display, Markdown

def test_query(query_text: str, patient_id: str = "12345678", apply_deidentification: bool = True):
    """
    Test function for interactive querying with de-identification options
    
    Args:
        query_text: The clinical question to ask
        patient_id: Patient identifier (default: "12345678")
        apply_deidentification: Whether to apply de-identification (default: True)
    
    Returns:
        Assistant response
    """
    if not assistant:
        return "❌ Clinical Assistant not initialized. Please set your OpenAI API key."
    
    header_md = f"""
### 🔍 Patient Query Test
- **Patient ID**: `{patient_id}`
- **Query**: *{query_text}*
- **De-identification**: `{"Enabled" if apply_deidentification else "Disabled"}`
"""

    display(Markdown(header_md))
    display(Markdown("---"))
    display(Markdown("### 🤖 Assistant Response"))

    response = assistant.process_clinical_query(query_text, patient_id, apply_deidentification)
    display(Markdown(f"```\n{response}\n```"))

    return response

🧪 Testing Clinical Assistant with Integrated De-identification


In [48]:


# Test with a sample patient ID (this will be de-identified)
test_patient_id = "1329b83e-ea69-d184-b4af-0d2a8e07896e"
query = sample_queries[6]
test_query(query, test_patient_id, apply_deidentification=True)


### 🔍 Patient Query Test
- **Patient ID**: `1329b83e-ea69-d184-b4af-0d2a8e07896e`
- **Query**: *Based on all available data, what is the comprehensive clinical picture?*
- **De-identification**: `Enabled`


---

### 🤖 Assistant Response

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Executing tool call: get_patient_observations


...

🔒 Redaction complete. PHI has been removed from the clinical note.
📄 Redacted note: Ms. Yaeko Ming Kshlerin, born 06/07/1999 (SSN [REDACTED], driver’s license [REDACTED], passport [REDACTED]), a 25‑year‑old white non‑Hispanic female from Oakes, North Dakota, presented to TOWNER COUNTY MEDICAL CENTER INC in Cando, ND for an ambulatory check‑up with referral for dental care. She underwent a digital intra‑oral radiography (DX, SOP 1.2.840.10008.5.1.4.1.1.1.3) of the internal part of the mouth (body structure code 700016008, procedure code 241046008); the encounter cost was $96.45, totaling a claim cost of $3,193.25, of which $2,554.60 was covered by the payer. Ms. Kshlerin’s medical expenses amount to $127,546.31 with coverage of $673,780.87 and an income of $63,061; her residence is [REDACTED] (Towner County, FIPS [REDACTED]). The visit was conducted by Dr. Shiloh Larson, general practice, and the encounter was classified as an “Encounter
Executing tool call: get_patient_immunizations


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


🔒 Redaction complete. PHI has been removed from the clinical note.
📄 Redacted note: Patient: [REDACTED], born on [REDACTED] (SSN: [REDACTED]), presented to [REDACTED] on [REDACTED] for a general wellness examination. Dr. [REDACTED] documented a routine physical, noting she received the Influenza seasonal injectable preservative free vaccine (code 140) at age [REDACTED], and discussed her healthcare expenses of $127,546.31 against coverage of $673,780.87, with a current income of $63,061; she resides at [REDACTED], North Dakota, 58324. Follow‑up is scheduled as needed; the total claim cost was $593.90, with $475.12 covered by her payer.
Executing tool call: get_patient_allergies
🔒 Redaction complete. PHI has been removed from the clinical note.
📄 Redacted note: Patient: [REDACTED], SSN: [REDACTED], born on [REDACTED] in [REDACTED], presented to [REDACTED] on [REDACTED] for an encounter for problem [REDACTED] related to [REDACTED] with moderate rhinoconjunctivitis and mild skin eruptions

```
Based on the available data for the patient with ID **1329b83e-ea69-d184-b4af-0d2a8e07896e**, here is a comprehensive clinical picture:

### Patient Demographics
- **Age**: 26 years old (born on June 7, 1999)
- **Gender**: Female
- **Ethnicity**: White, non-Hispanic
- **Residence**: Towner County, North Dakota
- **Income**: $63,061
- **Healthcare Expenses**: $127,546.31 against coverage of $673,780.87

### Clinical Encounters
1. **General Wellness Examination**: 
   - Conducted for routine physical assessment.
   - Received the Influenza seasonal injectable preservative-free vaccine.
   - Total claim cost: $593.90, with $475.12 covered by insurance.

2. **Medication Review**:
   - Encounter classified as an outpatient check-up.
   - Follow-up scheduled to review medication adherence and any new concerns.
   - Total claim cost: $1,257.75, fully covered by the payer.

3. **Respiratory Therapy**:
   - Received care for acute bronchitis.
   - Care plan initiated on December 21, 2015, and concluded on July 25, 2016.
   - Encounter cost: $96.45, fully covered by insurance.

4. **Dental Care Referral**:
   - Underwent digital intra-oral radiography.
   - Encounter cost: $96.45, with a total claim of $3,193.25, of which $2,554.60 was covered by the payer.

5. **Symptom Encounter**:
   - Presented with moderate rhinoconjunctivitis and mild skin eruptions.
   - Encounter cost: $483.55, with the patient responsible for the full cost.

### Medications
- **Acetaminophen 300 mg / Hydrocodone Bitartrate 5 mg**: 
   - Prescribed for pain management.
   - Total cost for medication: $97.29, fully covered by the payer.

### Procedures
- **Dental Imaging**: 
   - Digital intra-oral radiography performed as part of dental care.

### Immunizations
- **Influenza Vaccine**: 
   - Received as part of routine care.

### Allergies
- No specific allergy information was provided in the data.

### Summary
The patient is a 26-year-old white female with a history of respiratory issues (acute bronchitis) and current management for rhinoconjunctivitis. She has undergone various outpatient encounters, including routine wellness checks, medication reviews, and dental referrals. Her healthcare expenses are significant, but she has substantial insurance coverage. There are no recorded allergies, and she is currently on a pain management regimen involving acetaminophen and hydrocodone.

### Limitations
- Specific details regarding chronic conditions, allergies, and comprehensive medication history were not provided.
- Further information on recent laboratory tests or imaging studies would enhance the clinical picture.

If you have any specific questions or need further details, please let me know!
```

'Based on the available data for the patient with ID **1329b83e-ea69-d184-b4af-0d2a8e07896e**, here is a comprehensive clinical picture:\n\n### Patient Demographics\n- **Age**: 26 years old (born on June 7, 1999)\n- **Gender**: Female\n- **Ethnicity**: White, non-Hispanic\n- **Residence**: Towner County, North Dakota\n- **Income**: $63,061\n- **Healthcare Expenses**: $127,546.31 against coverage of $673,780.87\n\n### Clinical Encounters\n1. **General Wellness Examination**: \n   - Conducted for routine physical assessment.\n   - Received the Influenza seasonal injectable preservative-free vaccine.\n   - Total claim cost: $593.90, with $475.12 covered by insurance.\n\n2. **Medication Review**:\n   - Encounter classified as an outpatient check-up.\n   - Follow-up scheduled to review medication adherence and any new concerns.\n   - Total claim cost: $1,257.75, fully covered by the payer.\n\n3. **Respiratory Therapy**:\n   - Received care for acute bronchitis.\n   - Care plan initiated on 